In [47]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.pipeline import Pipeline

In [28]:
# dataSet 불러오기

feature = ['sepal_length', 'sepal_width', 'petal length', 'petal width', 'class']
iris = pd.read_csv("dataset/iris.csv", names=feature)
iris_v = iris.iloc[:,:4]
iris_v

,sepal_length,sepal_width,petal length,petal width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [75]:
# K_MEANS
from sklearn.cluster import KMeans

model = KMeans(n_clusters=3,init='k-means++')
model.fit(iris_v)
predict = pd.Series(model.predict(iris_v)) # predict로 iris['class']와 accuracy 측정
predict

0      2
1      2
2      2
3      2
4      2
      ..
145    0
146    1
147    0
148    0
149    1
Length: 150, dtype: int32

In [76]:
# K_MEANS crosstab

le = preprocessing.LabelEncoder() # 먼저 iris['class']를 형변환
target = pd.DataFrame(le.fit_transform(iris['class']))
ct = pd.crosstab(target[0], predict)
ct

col_0,0,1,2
0,,,
0,0,0,50
1,2,48,0
2,36,14,0


In [78]:
# 가장 정확도가 높은 쪽으로 매핑

predict_dict = {0:2, 1:1, 2:0}
predict = predict.map(predict_dict).astype('category')
predict

0      0
1      0
2      0
3      0
4      0
      ..
145    2
146    1
147    2
148    2
149    1
Length: 150, dtype: category
Categories (3, int64): [0, 1, 2]

In [82]:
# K_MEANS Accuracy Measurement
from sklearn.metrics import accuracy_score

accuracy_score(target, predict) # K_MEANS의 정확도는 약 89.33%

0.8933333333333333